In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from pandasql import sqldf

import matplotlib.pyplot as plt
import seaborn as sns

# **Задача**

По поведению пользователей на площадке rabota.ru предсказать, на какие вакансии пользователь в дальнейшем откликнется или позвонит.

В рамках задачи вам будет необходимо разработать модель, которая будет опираться на взаимодействия пользователей (соискателей) с вакансией. На выходе модель должна отдавать список из n рекомендованных вакансий, отсортированных по релевантности.


# **Данные**

## ***train_mfti.parquet*** – сырые данные, которые можно использовать для обучения модели
* event_date – дата взаимодейтсвия
* event_timestamp – timestamp взаимодействия в секундах 
* vacancy_id_ - id вакансии, с которой было взаимодействие
* cookie_id – id пользователя по его браузеру/ip/устройству
* user_id – id пользователя на сайте rabota.ru (есть только для зарегистрированных пользователей)
* event_type – тип взаимодействия


In [ ]:
df = pd.read_parquet('train_mfti.parquet')
display(df.head(3))
print(df.shape)

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


(12292588, 6)


## ***test_public_mfti.parquet*** – часть теста, с открытым таргетом, для проверки работоспособности решений.
* cookie_id - id пользователя по его браузеру/ip/устройству
* vacancy_id_ - список вакансий, на которые пользователь откликнулся или позвонил в течение месяца после окончания данных train


In [ ]:
df1_ = pd.read_parquet('test_public_mfti.parquet')
display(df1_.head(3))
print(df1_.shape)

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."


(772, 2)


## ***test_private_users_mfti.parquet*** – часть теста, с закрытым таргетом для итоговой проверки решений
* cookie_id - id пользователя по его браузеру/ip/устройству


In [ ]:
df2_ = pd.read_parquet('test_private_users_mfti.parquet')
display(df2_.head(3))
print(df2_.shape)

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf


(3086, 1)


## ***test_private_sample_submission_mfti.parquet*** – файл с примером предсказаний, который требуется получить по итогу хакатона 
* cookie_id - id пользователя по его браузеру/ip/устройству
* predictions – список из 5 id вакансий, которые модель предсказала как наиболее релеватные для данного пользователя


In [ ]:
df3_ = pd.read_parquet('test_private_sample_submission_mfti.parquet')
display(df3_.head(3))
print(df3_.shape)

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"


(3086, 2)


# **Метрика**
* Результаты модели будут оцениваться по метрике ***precision@5***. 
* При демонстрации своего решения по завершению хакатона, вам необходимо будет предоставить ***код для обучения и инференса модели, презентацию по проделанной работе и файл с предсказаниями*** для пользователей из списка test_private_users_mfti.parquet в формате файла test_private_sample_submission_mfti.parquet


# **Рекомендации по решению**
* В таких задачах, чаще всего, baseline моделью считается рекомендация самых популярных вакансий для всех пользователей. Попробуйте написать полный pipeline с использованием этой простой модели и отталкиваться от него в процессе ваших исследований
* Обратите внимания на то, что если пользователь каким-либо образом взаимодействовал с вакансией в обучающих данных, то для этого пользователя данной вакансии не будет в списке меток теста. Мы не хотим рекомендовать пользователю то, что он уже видел и с чем взаимодействовал.
* Постарайтесь не использовать файл test_public_mfti.parquet для постоянной валидации своих моделей, это может привести к переобучению. Вместо этого лучше подготовить свой валидационный датасет.


# **Решение** (baseline)

In [ ]:
# Смотрим, какие типы взаимодействий имеются
df['event_type'].unique()

array(['show_vacancy', 'preview_click_vacancy', 'click_contacts',
       'preview_click_contacts', 'click_favorite',
       'preview_click_favorite', 'preview_click_phone',
       'preview_click_response', 'click_phone', 'click_response'],
      dtype=object)

In [ ]:
# Выполним mapping для типов взаимодействий:
df['rating'] = df['event_type'].map({'show_vacancy': 0,
                                     'preview_click_vacancy': 1,
                                     'click_favorite': 2,
                                     'preview_click_favorite': 3,
                                     'click_response': 4,
                                     'preview_click_response': 5,
                                     'click_contacts': 6,
                                     'preview_click_contacts': 7,
                                     'click_phone': 8,
                                     'preview_click_phone': 9})

In [ ]:
# Количество значений в каждом столбце и тип данных
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 7 columns):
 #   Column           Non-Null Count     Dtype 
---  ------           --------------     ----- 
 0   event_date       12292588 non-null  object
 1   event_timestamp  12292588 non-null  int64 
 2   vacancy_id_      12292588 non-null  int64 
 3   cookie_id        12292588 non-null  object
 4   user_id          8711177 non-null   object
 5   event_type       12292588 non-null  object
 6   rating           12292588 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 656.5+ MB


In [ ]:
# Вывод количества уникальных id
print(f'{"Столбец:" : <15} {"Количество уникальных:"}')
for col_id in ('vacancy_id_', 'cookie_id', 'user_id'):
    print(f'{col_id.split(sep="_")[0] : <15} {df[col_id].nunique(dropna=False)}')

Столбец:        Количество уникальных:
vacancy         160167
cookie          330180
user            209336


In [ ]:
df['cookie_id'].value_counts()[:10]

924398d361a0454c8c30845c2b4c5747    4898
7d8cc5a0fef44378a2d90a237cda288e    3343
5934d5b8a0b348829d8efabe69c733eb    3315
b7dd2f20fdd6472ab62f8d86a739cd5d    3312
353b56c4fa6447d0ba0f08b42d86e51f    2795
6b3281f474314f96b75a7d1a06f09eed    2478
43cef1cac8f646a1adf620cb023fb311    2262
57013f363bb64956a65c4061c75e43e1    2143
54c29201d1eb4a9bb446090e631e948a    2120
784a284e0bd34228903d95cff1559ca4    2103
Name: cookie_id, dtype: int64

In [ ]:
# Интересный момент, выглядит будто "накрутка"
df[df['cookie_id'] == '57013f363bb64956a65c4061c75e43e1']['vacancy_id_'].value_counts()

164246    1199
201312     788
219158     149
207169       3
257747       2
177710       2
Name: vacancy_id_, dtype: int64

In [ ]:
# ТОП-5 cookie по количеству вакансий, с которыми они взаиимодействовали
df.groupby(by='cookie_id').nunique()['vacancy_id_'].sort_values(ascending=False)[:5]

cookie_id
924398d361a0454c8c30845c2b4c5747    4541
784a284e0bd34228903d95cff1559ca4    1228
728c61d1db294cb59e6032a65706b964    1106
6b3281f474314f96b75a7d1a06f09eed     961
7d8cc5a0fef44378a2d90a237cda288e     826
Name: vacancy_id_, dtype: int64

In [ ]:
#df.groupby(by='cookie_id').nunique()

## Оценки вакансий

Создадим новую таблицу:
1. vacancy_id - идентификатор вакансии
2. interactions - количевство взаимодействий [количество всех взаимодействий]
3. total_score - суммарный балл по кликам [сумма баллов в зависимости от рейтинга клика]
4. reach - охват cookie [количество уникальных cookie, взаимодействующих как-либо с вакансией]
5. ER - доля вовлеченности [суммарный балл / количество всех взаимодействий]
6. ERR - доля вовлеченности по охвату [суммарный балл / охват]
7. max_score - максимальный скоринг [сумма максимального рейтинга взаимодействия от каждого cookie / количество cookie]

In [ ]:
query = """
    SELECT
        d.vacancy_id,
        d.interactions,
        d.total_score,
        d.reach, 
        CAST(d.total_score AS float) / d.interactions AS ER,
        CAST(d.total_score AS float) / d.reach AS ERR,
        CAST(SUM(sc.max_rating) AS float) / d.reach AS max_score
    FROM  (
        SELECT
            vacancy_id_ AS vacancy_id,
            COUNT(rating) AS interactions,
            SUM(rating) AS total_score,
            COUNT(DISTINCT cookie_id) AS reach
        FROM 
            df
        GROUP BY 
            vacancy_id_
    ) d
    LEFT JOIN (
        SELECT
            vacancy_id_,
            cookie_id,
            MAX(rating) AS max_rating
        FROM 
            df
        GROUP BY 
            vacancy_id_, cookie_id
    ) sc
    ON d.vacancy_id = sc.vacancy_id_
    GROUP BY 
        d.vacancy_id
    
        """
data = sqldf(query)

In [ ]:
data.head(3)

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score
0,100001,49,50,20,1.020408,2.500000,1.950000
1,100002,847,866,224,1.022432,3.866071,2.044643
2,100003,60,40,29,0.666667,1.379310,1.206897


In [ ]:
# Копия, чтобы быстро откатиться назад
v_metrics = data.copy()

In [ ]:
# Ввожу новую метрику оценки
v_metrics['my_score'] = np.log(v_metrics['interactions']) * (v_metrics['max_score'])

In [ ]:
# Ввожу новую метрику оценки 2
v_metrics['my_score2'] = np.log(v_metrics['reach']) * (v_metrics['max_score'])

In [ ]:
# Ввожу новую метрику оценки 3
v_metrics['my_score3'] = v_metrics['reach'] * (v_metrics['max_score'])

In [ ]:
v_metrics.describe()

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score,my_score,my_score2,my_score3
count,160167.00000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000,160167.000000
mean,180084.00000,76.748569,69.059257,29.210668,1.418143,2.924368,2.051753,5.162810,3.698908,47.392472
std,46236.37462,484.159623,410.200334,175.746648,1.466716,3.244840,1.392563,3.347428,2.596214,285.754177
min,100001.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,140042.50000,10.000000,8.000000,4.000000,0.612245,1.448276,1.200000,2.708050,1.732868,6.000000
50%,180084.00000,24.000000,20.000000,10.000000,0.866667,2.142857,1.640000,4.969813,3.532677,15.000000
75%,220125.50000,59.000000,54.000000,23.000000,1.333333,3.285714,2.263158,7.278045,5.364159,38.000000
max,260167.00000,59105.000000,44962.000000,18662.000000,9.000000,197.000000,9.000000,29.555835,25.749141,28130.000000


In [ ]:
# Похоже на попытку "поднять" вакансию в ТОП
v_metrics[(v_metrics['ERR'] > 100)]

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score,my_score,my_score2,my_score3
45597,145598,22,138,1,6.272727,138.0,8.0,24.728340,0.0,8.0
63009,163010,36,106,1,2.944444,106.0,7.0,25.084633,0.0,7.0
80907,180908,31,197,1,6.354839,197.0,8.0,27.471898,0.0,8.0
126336,226337,16,103,1,6.437500,103.0,9.0,24.953299,0.0,9.0
129448,229449,18,103,1,5.722222,103.0,8.0,23.122974,0.0,8.0


In [ ]:
# Выбирать рекомендацию по оценке "my_score" не совсем правильно,
# т.к. много записей с высоким баллом, но с малым количеством пользователей
v_metrics[v_metrics['my_score'] > 25]

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score,my_score,my_score2,my_score3
1395,101396,4297,7005,823,1.630207,8.511543,3.318348,27.760208,22.275921,2731.0
16822,116823,40023,41019,10901,1.024886,3.762866,2.492157,26.409907,23.168608,27167.0
63009,163010,36,106,1,2.944444,106.000000,7.000000,25.084633,0.000000,7.0
74952,174953,21864,24577,6237,1.124085,3.940516,2.654161,26.521957,23.192731,16554.0
80381,180382,12650,13430,3340,1.061660,4.020958,2.717665,25.669464,22.050387,9077.0
80907,180908,31,197,1,6.354839,197.000000,8.000000,27.471898,0.000000,8.0
82869,182870,29649,39097,7871,1.318662,4.967221,2.870283,29.555835,25.749141,22592.0
84066,184067,468,1374,80,2.935897,17.175000,4.687500,28.820945,20.540750,375.0
135107,235108,65,126,4,1.938462,31.500000,6.000000,25.046324,8.317766,24.0
145951,245952,3490,5295,700,1.517192,7.564286,3.138571,25.603389,20.561034,2197.0


In [ ]:
# По-моему, метрика "my_score2" хорошо отражает весомые взаимодействия
# и количество пользователей
v_metrics[v_metrics['my_score2'] > 20]

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score,my_score,my_score2,my_score3
1395,101396,4297,7005,823,1.630207,8.511543,3.318348,27.760208,22.275921,2731.0
16822,116823,40023,41019,10901,1.024886,3.762866,2.492157,26.409907,23.168608,27167.0
74952,174953,21864,24577,6237,1.124085,3.940516,2.654161,26.521957,23.192731,16554.0
80381,180382,12650,13430,3340,1.061660,4.020958,2.717665,25.669464,22.050387,9077.0
82869,182870,29649,39097,7871,1.318662,4.967221,2.870283,29.555835,25.749141,22592.0
84066,184067,468,1374,80,2.935897,17.175000,4.687500,28.820945,20.540750,375.0
93330,193331,10820,12146,3437,1.122551,3.533896,2.456794,22.821529,20.004085,8444.0
107422,207423,38607,38884,11988,1.007175,3.243577,2.167251,22.888743,20.354084,25981.0
145951,245952,3490,5295,700,1.517192,7.564286,3.138571,25.603389,20.561034,2197.0


In [ ]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
# method='first' для того, чтобы у каждой вакансии было уникальное значение в рейтинге
v_metrics['my_score_rank'] = v_metrics[['my_score', 'reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
                                    .rank(method='first',ascending=False).astype(int)

In [ ]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
v_metrics['my_score2_rank'] = v_metrics[['my_score2', 'reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
# Ранжирую вакансии по новой оценке (по охвату, если одинаковый балл и т.д.)
v_metrics['my_score3_rank'] = v_metrics[['my_score3', 'reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['max_score_rank'] = v_metrics[['max_score', 'reach', 'interactions', 'total_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['interactions_rank'] = v_metrics[['interactions', 'reach', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['total_score_rank'] = v_metrics[['total_score', 'reach', 'interactions', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['reach_rank'] = v_metrics[['reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['ER_rank'] = v_metrics[['ER', 'reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics['ERR_rank'] = v_metrics[['ERR', 'reach', 'interactions', 'total_score', 'max_score']].apply(tuple,axis=1)\
             .rank(method='first',ascending=False).astype(int)

In [ ]:
v_metrics.sort_values('reach_rank').head(5)

,vacancy_id,interactions,total_score,reach,ER,ERR,max_score,my_score,my_score2,my_score3,my_score_rank,my_score2_rank,my_score3_rank,max_score_rank,interactions_rank,total_score_rank,reach_rank,ER_rank,ERR_rank
160153,260154,59105,44962,18662,0.760714,2.409281,1.507341,16.561264,14.823561,28130.0,457,122,1,91438,1,1,1,96460,68624
98113,198114,50187,38400,16674,0.765138,2.302987,1.658510,17.950905,16.123383,27654.0,225,62,2,78991,2,5,2,95638,73059
103403,203404,45634,32158,16284,0.704694,1.974822,1.445161,15.504276,14.015081,23533.0,786,179,6,98102,3,7,3,105892,91744
11504,111505,35095,23346,15088,0.665223,1.547322,1.284133,13.439498,12.355486,19375.0,2184,466,10,113742,9,14,4,112888,114710
102607,202608,41620,32483,14524,0.780466,2.236505,1.658152,17.636662,15.890996,24083.0,264,70,5,79002,4,6,5,93119,75931


In [ ]:
1/0
v_metrics.to_parquet('vacancy_metrics.parquet')

In [ ]:
def precision_n(predictions, df_test_list_top, k=5):
    
    # Количество рекомендаций по k вакансий:
    n_rec_k = len(df_test_list_top)*k
    
    # Подсчет валидных вакансий:
    matches = 0
    for row in range(len(predictions)):
        row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vacancy_id_'][row]))  
        matches += len(row_match)
    
    precisions = matches / n_rec_k if n_rec_k != 0 else 0
    return precisions

In [ ]:
def top_vacancy(df1=df1_, df2=v_metrics, df3=df, rank='my_score2_rank'):
    
    """
    df1 >>> датафрейм, из которого нужно получить список "cookie_id" для рекомендаций
    df2 >>> датафрейм, с рангами
    df3 >>> датафрейм (лог-файл), здесь идет поиск вакансий, с которыми 
    пользователь уже взаимодействовал 
    rank >>> название столбца df2 с рейтингом вакансий
    """
    
    # Выбираю список cookie, для которых рекомендуются вакансии
    test_list = list(df1['cookie_id'])

    # Новый датафрейм с сookie_id и 5-ю рекомендуемыми вакансиями
    result = pd.DataFrame(columns=['cookie_id', 'top_recommendations'])
    
    # Для каждого пользователя свои рекомендации
    for cookie in test_list:
        # Список с вакансиями, с которыми пользователь уже взаимодействовал
        not_recommend = df3[df3['cookie_id'] == cookie]['vacancy_id_'].unique()
        # Список из 5 рекомендуемых вакансий
        top5 = df2[~df2['vacancy_id'].isin(not_recommend)].sort_values(rank).head(5)['vacancy_id'].tolist()
        # Запись результатов
        result = pd.concat([result, pd.DataFrame({'cookie_id': cookie,
                                                  'top_recommendations': [top5]})], ignore_index=True)
        
    return precision_n(result, df1)

In [ ]:
for col in v_metrics.loc[:, 'my_score_rank':'ERR_rank'].columns:
    print(f'precision@5: {top_vacancy(df1_, v_metrics, df, col): <23} по "{col}"')

precision@5: 0.007512953367875648    по "my_score_rank"
precision@5: 0.008031088082901554    по "my_score2_rank"
precision@5: 0.01917098445595855     по "my_score3_rank"
precision@5: 0.0                     по "max_score_rank"
precision@5: 0.019689119170984457    по "interactions_rank"
precision@5: 0.021502590673575128    по "total_score_rank"
precision@5: 0.02072538860103627     по "reach_rank"
precision@5: 0.0                     по "ER_rank"
precision@5: 0.0                     по "ERR_rank"


Вывод:
При проставлении баллов по взаимодействиям пользователя с вакансиями:

* 'show_vacancy': 0,
* 'preview_click_vacancy': 1,
* 'click_favorite': 2,
* 'preview_click_favorite': 3,
* 'click_response': 4,
* 'preview_click_response': 5,
* 'click_contacts': 6,
* 'preview_click_contacts': 7,
* 'click_phone': 8,
* 'preview_click_phone': 9

В качестве baseline решения (рекомендации самых популярных вакансий) были проверены следующие оценки:
1. interactions - количевство взаимодействий [количество всех взаимодействий]
2. total_score - суммарный балл по кликам [сумма баллов в зависимости от рейтинга клика]
3. reach - охват [количество уникальных cookie, взаимодействующих как-либо с вакансией]
4. ER - доля вовлеченности [суммарный балл / количество взаимодействий]
5. ERR - доля вовлеченности по охвату [суммарный балл / охват]
6. max_score - максимальный скоринг [макимальный балл от каждого cookie / количество cookie]
7. my_score - ln(interactions) * (max_score)
8. my_score2 - ln(reach) * (max_score)
9. my_score3 - reach * (max_score)

Результаты:
precision@5: 0.007512953367875648    по my_score
precision@5: 0.008031088082901554    по my_score2
precision@5: 0.01917098445595855     по my_score3
precision@5: 0.0                     по max_score
precision@5: 0.019689119170984457    по interactions
precision@5: 0.021502590673575128    по total_score
precision@5: 0.02072538860103627     по reach
precision@5: 0.0                     по ER
precision@5: 0.0                     по ERR

Выводы:
1. Неудивительно, что по max_score, ER, ERR решения привели к "нулевым" результатам, т.к. в ТОП'ах по этим оценкам вакансии, у которых один или несколько пользователей, но с "высокооцениваемыми" кликами (накрутка будет обеспечена).
2. Лучшие показатели по метрике precision@5 показали рекомендации по interactions, total_score, reach. В целом, логичный результат, но у такого подхода есть минус - вакансии, находящиеся в ТОП'aх показывают намного чаще, тем самым объявления набирают свои баллы быстрее других. Остальные же вакансии (хорошие или плохие) не отображаются пользователям, если это не "конкретный поиск". Разрыв между популярными и непопулярными будет увеличиваться.
3. Несмотря на то, что результат по my_score2 = ln(reach) * (max_score) не стал лучшим. Рекомендации по этой оценке учитывают и "важность" кликов (причем без выбросов т.к. max_score=[0, 9]), и охват пользователей (логарифм используется для того, чтобы вакансии, которые рекомендовали всем подряд и у них явное преимущество по количеству пользователей, в конечном счете могли опуститься ниже тех, у которых не очень много пользователей, но хороший показатель по max_score).